In [1]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import ast
from IPython.display import clear_output
import random

In [2]:
# Steg 1: Ladda dataset
original_training_data = pd.read_csv("training_knn.csv")
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
encode_columns = ['Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles']
ID_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", "ID_Secondary Muscles"]

important_data.head()

,Exercise Name,Main_muscle,Mechanics,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Military Press,Shoulder,Compound,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Military Press: Seated,Shoulder,Compound,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Front Raise: Alternating,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [3]:
def encoded_data(specific_muscle):
    new_data = important_data.copy()
    target_muscle_mapping = []  # List to store muscle names at correct indices
    
    for column in encode_columns:
        new_data.loc[:, column] = new_data[column].fillna("")
        new_data.loc[:, column] = new_data[column].apply(lambda x: x.split(", ") if x else [])
        
        # Multi-label binarization
        mlb = MultiLabelBinarizer()
        binary_matrix = mlb.fit_transform(new_data[column])
        
        # Save muscle names in a list if the column is "Target_Muscles"
        if column == "Target_Muscles":
            target_muscle_mapping = list(mlb.classes_)        
        new_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()
        new_data = new_data.drop(columns=[column])
    finished_data = pd.DataFrame()
    for i in range(len(target_muscle_mapping)):
        muscle_index=[]
        if target_muscle_mapping[i].replace(",","") == specific_muscle:
            muscle_index.append(i)
        for index in muscle_index:
            temporary_dataset = new_data[new_data["ID_Target_Muscles"].apply(lambda x: x[index] == 1)]
            finished_data = pd.concat([finished_data, temporary_dataset], ignore_index=True)
    return finished_data


In [4]:
def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    """
    Adjust scores based on the user's fitness level and exercise difficulty.
    """
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        factor = 1 - (abs(user_fitness_level - difficulty) / max_difference) ** 2
        factor = max(0.5, factor)
        adjusted_scores.append(pred * factor)
    return adjusted_scores

def knn(predict_data, user_fitness_level,All_Exercises):
    """
    Perform KNN classification and adjust the predicted scores.
    """
    training_data = original_training_data.copy()
    for col in ID_columns:
        def safe_literal_eval(value):
            try:
                # Ensure the value is parsed correctly
                if isinstance(value, str):
                    return np.array(ast.literal_eval(value))
                elif isinstance(value, list):
                    return np.array(value)
                else:
                    return np.array([])  # Return empty array for invalid entries
            except (ValueError, SyntaxError):
                return np.array([])  # Handle malformed data gracefully

        training_data[col] = training_data[col].apply(safe_literal_eval)

    # Flatten the training data
    flattened_features = []
    for col in ID_columns:
        arrays = np.stack(training_data[col].values)
        array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
        flattened_features.append(array_df)

    data_flattened = pd.concat([training_data.drop(ID_columns, axis=1)] + flattened_features, axis=1)

    X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
    y = data_flattened["score"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_scaled, y)

    # Process predict_data
    for col in ID_columns:
        predict_data[col] = predict_data[col].apply(safe_literal_eval)

    # Flatten the test data
    flattened_features_test = []
    for col in ID_columns:
        arrays_test = np.stack(predict_data[col].values)
        array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
        flattened_features_test.append(array_df_test)

    data_test_flattened = pd.concat([predict_data.drop(ID_columns, axis=1)] + flattened_features_test, axis=1)

    X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
    X_test_scaled = scaler.transform(X_test)

    y_pred_test = knn.predict(X_test_scaled)

    # Adjust scores using the function
    exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
    adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

    data_test_flattened["Predicted Score"] = adjusted_scores_test

    sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)
    sorted_results = sorted_results[~sorted_results['Exercise Name'].isin(All_Exercises[i])]
    return sorted_results["Exercise Name"].iloc[0]


In [5]:
# Välj relevanta kolumner för klustring
clustered_features = exercise_data[['Main_muscle', 'Target_Muscles']]

# Använd OneHotEncoder för att omvandla dessa kolumner till numeriska värden
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(clustered_features)

# Använd K-means för att klustra övningarna
kmeans = KMeans(n_clusters=10, random_state=42)  # Välj antal kluster till 11
exercise_data['Cluster'] = kmeans.fit_predict(encoded_features)

clustered_exercises = exercise_data.groupby('Cluster')['Exercise Name'].apply(list).to_dict()

In [6]:
# Display the list of main muscles with numbers
def display_muscles(muscles):
    print("Select one or more main muscles from the list:")
    for index, muscle in enumerate(muscles, start=1):
        print(f"{index}. {muscle}")

# Select main muscles and filter target muscles
def select_MainMuscles(data):
    available_muscles = data['Main_muscle'].unique().tolist()  # List of available main muscles
    selected_muscles = set()  # Set to keep track of selected muscles for the current day

    selecting = True
    while selecting:
        # Display available main muscles
        print("Select one or more main muscles from the list:")
        for index, muscle in enumerate(available_muscles, start=1):
            print(f"{index}. {muscle}")

        choice = input("Enter the number of the muscle you want to select (or 'q' to finish selection): ")

        if choice.lower() == 'q':
            selecting = False
        
        else:
            try:
                choice_index = int(choice) - 1
                if 0 <= choice_index < len(available_muscles):
                    selected_muscle = available_muscles[choice_index]
                    if selected_muscle in selected_muscles:
                        print(f"You have already selected: {selected_muscle}. Please choose another muscle.")
                    else:
                        print(f"You have selected: {selected_muscle}")
                        selected_muscles.add(selected_muscle)  # Add to the set to prevent future selections for today
                else:
                    print("Invalid choice, please try again.")
            except ValueError:
                print("Please enter a valid number.")

    return selected_muscles  # Return the set of selected main muscles for the day

# Filter target muscles based on selected main muscles
def filter_TargetMuscles(data, selected_muscles):
    target_muscles = []  # List to store target muscles
    for muscle in selected_muscles:
        filtered_data = data[data['Main_muscle'] == muscle]
        for _, target in filtered_data.iterrows():
            target_name = target['Target_Muscles']  # Corrected column name
            if target_name not in target_muscles:  # Check for duplicates
                target_muscles.append(target_name)  # Append target muscles to the list

    return target_muscles  # Return the list of target muscles

# Function to handle pre-planned workouts
def PREplanned_workout():
    print("Select a pre-planned workout type:")
    print("1. Push")
    print("2. Pull")
    print("3. Leg")
    print("4. Whole Body")

    while True:
        choice = input("Enter the number of the workout type you want to select: ")

        try:
            choice_index = int(choice)
            if choice_index == 1:  # Push
                main_muscles = ['Chest', 'Shoulder', 'Triceps']
                print("You have selected: Push")
                return main_muscles
            elif choice_index == 2:  # Pull
                main_muscles = ['Back', 'Biceps', 'Forearm']
                print("You have selected: Pull")
                return main_muscles
            elif choice_index == 3:  # Leg
                main_muscles = ['Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Leg")
                return main_muscles
            elif choice_index == 4:  # Whole Body
                main_muscles = ['Chest', 'Back', 'Shoulder', 'Biceps', 'Triceps', 'Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Whole Body")
                return main_muscles
            else:
                print("Invalid choice, please try again.")
        except ValueError:
            print("Please enter a valid number.")

# Function to select difficulty level
def select_difficulty():
    print("Select difficulty level")
    while True:
        difficulty = input("Select difficulty level (1-5): ")
        try:
            difficulty = int(difficulty)
            if 1 <= difficulty <= 5:
                print(f"You have selected difficulty level: {difficulty}")
                return difficulty  # Return the selected difficulty level
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")

# Dictionary to store workouts for each day of the week
weekly_workouts = {}

# List of days in a week
days_of_week = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

# Ask for difficulty level once for the whole week
difficulty = select_difficulty()  # Get the difficulty level for the week

for day in days_of_week:
    
    print(f"{day}:")
    print("Select workout type:")
    print("1. Pre-planned")
    print("2. Hybrid")
    print("3. Rest Day")  # Added Rest Day option here

    while True:  # Loop until a valid choice is made
        workout_choice = input("Enter the number of the workout type you want to select: ")
        
        try:
            workout_choice_index = int(workout_choice)  # Try to convert input to an integer
            if workout_choice_index == 1:  # Pre-planned
                main_muscles = PREplanned_workout()
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 2:  # Hybrid
                main_muscles = select_MainMuscles(exercise_data)
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 3:  # Rest Day
                print("Rest day selected. No workout planned.")
                weekly_workouts[day] = []  # Set to an empty list for Rest Day
                break  # Exit the inner loop
            else:
                print("Invalid choice, please try again.")  # Prompt for valid input again
        except ValueError:
            print("Please enter a valid number.")  # Handle non-integer inputs
    
    # Collect all unique target muscles for the selected main muscles
    if workout_choice_index != 3:  # Only filter target muscles if it's not a Rest Day
        selected_targets = filter_TargetMuscles(exercise_data, main_muscles)

        # Store the selected target muscles in the dictionary
        if selected_targets:
            weekly_workouts[day] = selected_targets  # Store as a dictionary
    clear_output()


In [7]:
# Print the weekly workouts
print("Weekly Workouts:")
for day, exercises in weekly_workouts.items():
    muscles = {muscle.strip() for exercise in exercises for muscle in exercise.split(',') if muscle.strip() != ""}
    weekly_workouts[day] = list(muscles)
print(weekly_workouts)
print(f"Difficulty level for the week is set to: {difficulty}")

Weekly Workouts:
{'MONDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'TUESDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'WEDNESDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'THURSDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'FRIDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'SATURDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus'], 'SUNDAY': ['Gastrocnemius', 'Tibialis Anterior', 'Hamstrings', 'Quadriceps', 'Soleus', 'Gluteus Maximus']}
Difficulty level for the week is set to: 3


In [8]:
All_Exercises = {"MONDAY" :[], 
"TUESDAY": [], 
"WEDNESDAY": [], 
"THURSDAY": [], 
"FRIDAY": [], 
"SATURDAY": [], 
"SUNDAY": []}
for i in weekly_workouts:
    worked_muscles = []
    for k in weekly_workouts[i]:
        if worked_muscles.count(k) <= 2:
            k = k.replace(",", "") 
            predic_data = encoded_data(k)  
            exercise = knn(predic_data, difficulty, All_Exercises) 
            row = exercise_data[exercise_data['Exercise Name'] == exercise]  
            primary_muscles = row['Target_Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in primary_muscles if muscle.strip() != ""])
            secondary_muscles = row['Secondary Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in secondary_muscles if muscle.strip() != ""])
            All_Exercises[i].append(exercise)
print(All_Exercises)



In [9]:
# Funktion för att hitta en ny övning från samma kluster
def get_new_exercise(exercise_name, clustered_exercises):
    # Hitta klustret för övningen
    for cluster, exercises in clustered_exercises.items():
        if exercise_name in exercises:
            # Filtrera bort nuvarande övning och välj en ny
            other_exercises = [ex for ex in exercises if ex != exercise_name]
            return random.choice(other_exercises) if other_exercises else exercise_name
    return exercise_name  # Om inget kluster hittas, returnera samma övning

def adjust_schedule(all_exercises, clustered_exercises):
    days = list(all_exercises.keys())
    num_days = len(days)

    for i in range(num_days):  # Gå igenom alla dagar
        previous_day = days[i - 1] if i > 0 else None
        current_day = days[i]
        next_day = days[(i + 1) % num_days] if i < num_days - 1 else None  # Hantera söndag -> måndag

        unique_exercises = set()  # Hålla koll på övningar inom samma dag
        adjusted_day_exercises = []

        for exercise in all_exercises[current_day]:
            # Förhindra dubletter inom samma dag
            if exercise in unique_exercises:
                # Byt ut övningen mot en ny från samma kluster
                new_exercise = get_new_exercise(exercise, clustered_exercises)
                while new_exercise in unique_exercises:  # Säkerställ att den nya också är unik
                    new_exercise = get_new_exercise(new_exercise, clustered_exercises)
                adjusted_day_exercises.append(new_exercise)
                unique_exercises.add(new_exercise)
            else:
                adjusted_day_exercises.append(exercise)
                unique_exercises.add(exercise)

        # Uppdatera övningar för dagen med unika övningar
        all_exercises[current_day] = adjusted_day_exercises

        # Kontrollera överlappning med föregående dag
        if previous_day:
            overlapping_exercises = set(all_exercises[previous_day]) & set(all_exercises[current_day])
            for exercise in overlapping_exercises:
                new_exercise = get_new_exercise(exercise, clustered_exercises)
                while new_exercise in all_exercises[current_day]:
                    new_exercise = get_new_exercise(new_exercise, clustered_exercises)
                all_exercises[current_day] = [
                    new_exercise if ex == exercise else ex for ex in all_exercises[current_day]
                ]

        # Kontrollera överlappning med nästa dag (för söndag -> måndag)
        if next_day:
            overlapping_exercises = set(all_exercises[next_day]) & set(all_exercises[current_day])
            for exercise in overlapping_exercises:
                new_exercise = get_new_exercise(exercise, clustered_exercises)
                while new_exercise in all_exercises[current_day]:
                    new_exercise = get_new_exercise(new_exercise, clustered_exercises)
                all_exercises[current_day] = [
                    new_exercise if ex == exercise else ex for ex in all_exercises[current_day]
                ]

    return all_exercises

# Justera schemat för att undvika upprepade övningar
All_Exercises = adjust_schedule(All_Exercises, clustered_exercises)


In [10]:
data = []

for day, exercises in All_Exercises.items():
    data.append({'Day': day, 'Exercise': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})  # Blank row for the day name
    for exercise in exercises:
        information = exercise_data[exercise_data['Exercise Name'] == exercise]
        data.append({
            'Day': '',  # Leave the day column empty for subsequent exercises
            'Exercise': exercise,
            'Main Muscle': information['Main_muscle'].iloc[0],
            'Equipment': information['Equipment'].iloc[0],
            'Preparation': information['Preparation'].iloc[0],
            'Execution': information['Execution'].iloc[0]
        })
    data.append({'Day': '', 'Exercise': '', 'Main Muscle': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head(70)

,Day,Exercise,Equipment,Preparation,Execution,Main Muscle
0,MONDAY,,,,,NaN
1,,Lying Calf Press,Sled,Position sled away from platform. Lie supine o...,Push sled away from platform by extending ankl...,Calves
2,,45° Calf Press,Lever (plate loaded),Sit on seat with back on padded support. Place...,Push platform by extending ankles as far as po...,Calves
3,,Straight Hip Leg Curl,Suspended,Sit on floor facing suspension trainer in low ...,"Bend knees, pulling heels toward rear end whil...",Hamstring
4,,Split Squat,Cable,"Facing low pulley, kneel to attach cable belt ...",Squat down by flexing knee and hip of front le...,Glutes
5,,Seated Calf Raise,Lever (plate loaded),Sit on seat facing lever. Place forefeet on pl...,Lower heels by bending ankles until calves are...,Calves
6,,Seated Pronation,Dumbbell,"Sit next to elevated surface, approximately ar...",Rotate dumbbell so thumb turns downward (prona...,Forearm
7,,,,,,
8,TUESDAY,,,,,NaN
9,,Calf Extension,Lever (plate loaded),Sit on seat and position forefeet on lever pla...,Push lever by extending ankle as far as possib...,Calves


In [11]:
output_file = 'final_scheme.xlsx'
df.to_excel(output_file, index=False)